In [ ]:
#! pip install pymorphy3

In [ ]:
import pandas as pd
import numpy as np
from gensim.downloader import load, info
from gensim.models import Word2Vec
from pymorphy3 import MorphAnalyzer

In [ ]:
morph = MorphAnalyzer()

In [ ]:
rv = load('word2vec-ruscorpora-300') # - скачивание предобученной модел

Было проведенно не большое изучение данных, далее в коде остались лиш основные моменты

Данные в ручную импортировались в csv и обратно путм имрота/экспорта  в access

In [ ]:
mtr = pd.read_csv("MTR.csv", sep=";", names=["код СКМТР", "Наименование", "Маркировка", "Регламенты (ГОСТ/ТУ)", "Параметры", "Базисная", "ОКПД2"])

In [ ]:
mtr.head(5)

,код СКМТР,Наименование,Маркировка,Регламенты (ГОСТ/ТУ),Параметры,Базисная,ОКПД2
0,8559150569,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-182 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
1,8559150570,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-188 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
2,8559150571,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-194 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
3,8559150572,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,128-112-152 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131
4,8559150573,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,128-112-158 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131


In [ ]:
mtr['ОКПД2'].nunique()

3630

In [ ]:
mtr['ОКПД2'].value_counts()[:1000]

,count
ОКПД2,
14.12.30.132,62481
14.12.30.131,44862
14.12.11.120,30723
14.12.21.120,25996
30.20.40.149,25071
...,...
10.85.11.000,53
25.73.40.163,52
32.40.39.192,52


Формирование групп проходило отдельно для каждой подкатегории ОКПД. Из каждой строки выделялось ведущие слово, лучше всего характеризующие товар. Затем ведущие слова формировались в общий список и сортировались по популярности. После чего для каждой строки была проведена проверка, на сколько по смыслу разные ведущие слово сопоставимы данному товару, исходя из которой лучший вариант становился категорией


Далее приведн код в котором ведущие слово выделяется как первое слово название, не смотря на простоту подхода, оно точно описывает абсолютное большинство товара.

Проверка схожести ведущего слова и товара осуществляется путём сравнения косинусного сходства векторов.

Для проверки косинусного сходства применялись методы nlp с загрузкой пред обученных моделей и обработкой текстовых данных.


In [ ]:
list_okpd = list(mtr['ОКПД2'].unique())[:100]

In [ ]:
def normalaz(words):
  norm_word = []
  for word in words:
    word = word.lower()

#    if '-' in word:
#      word,b = word.split('-')

    world = morph.normal_forms(word)
    norm_word.append(word)

  return norm_word

In [ ]:
def check(name,group_name):
  group = []
  for n in name:
    flag = 1
    #print(n)
    if n +'_NOUN' in rv.key_to_index:
      for g in group_name:
        #print(g)
        if g + '_NOUN' in rv.key_to_index:
          #print(rv.similarity(n +'_NOUN', g+'_NOUN'),n,g)
          if rv.similarity(n +'_NOUN', g+'_NOUN') > 0.6:
            group.append(g)
            flag = 0
            break
    if flag == 1:
      group.append(n)
  return group


In [ ]:
mtr['ОКПД2'] = mtr['ОКПД2'].astype(str)
mtr_group = pd.DataFrame()

for okpd in list_okpd :
  produkt  = mtr[mtr['ОКПД2'].str.contains(okpd)]

  print(okpd)
  name= pd.DataFrame(produkt['Наименование'])
  first_words = name['Наименование'].str.split().str[0]

  word_counts = pd.Series(first_words).value_counts()
  sorted_words = word_counts.index.tolist()

  group_name = normalaz(sorted_words)

  normal_name = normalaz(pd.DataFrame(name['Наименование'].str.split().str[0])['Наименование'])

  first_words = pd.DataFrame(first_words)
  first_words['normal_name'] = list(normal_name)

  list_group = check(first_words['normal_name'],group_name)
  first_words['group'] = list_group

  data = pd.concat([produkt, first_words[['group']]], axis=1)
  mtr_group = pd.concat([mtr_group, data], ignore_index=True)
  print(mtr_group.columns)
  print(mtr_group.shape)

mtr = pd.merge(mtr, mtr_group[['код СКМТР', 'group']], on='код СКМТР', how='left')


14.12.30.131
Index(['код СКМТР', 'Наименование', 'Маркировка', 'Регламенты (ГОСТ/ТУ)',
       'Параметры', 'Базисная', 'ОКПД2', 'group'],
      dtype='object')
(44862, 8)
30.20.40.149
Index(['код СКМТР', 'Наименование', 'Маркировка', 'Регламенты (ГОСТ/ТУ)',
       'Параметры', 'Базисная', 'ОКПД2', 'group'],
      dtype='object')
(69933, 8)
31.01.12.190
Index(['код СКМТР', 'Наименование', 'Маркировка', 'Регламенты (ГОСТ/ТУ)',
       'Параметры', 'Базисная', 'ОКПД2', 'group'],
      dtype='object')
(71388, 8)
58.19.13.120
Index(['код СКМТР', 'Наименование', 'Маркировка', 'Регламенты (ГОСТ/ТУ)',
       'Параметры', 'Базисная', 'ОКПД2', 'group'],
      dtype='object')
(71705, 8)
30.99.10.190
Index(['код СКМТР', 'Наименование', 'Маркировка', 'Регламенты (ГОСТ/ТУ)',
       'Параметры', 'Базисная', 'ОКПД2', 'group'],
      dtype='object')
(72089, 8)
25.73.40.290
Index(['код СКМТР', 'Наименование', 'Маркировка', 'Регламенты (ГОСТ/ТУ)',
       'Параметры', 'Базисная', 'ОКПД2', 'group'],
      d

In [ ]:
mtr.head()

,код СКМТР,Наименование,Маркировка,Регламенты (ГОСТ/ТУ),Параметры,Базисная,ОКПД2,group
0,8559150569,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-182 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,сорочка
1,8559150570,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-188 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,сорочка
2,8559150571,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-194 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,сорочка
3,8559150572,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,128-112-152 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,сорочка
4,8559150573,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,128-112-158 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,сорочка


Датасет после получения групп

## Параметры


> Добавить блок с цитатой



Задача отбора признаков имела под собой множество подводных камней.


Основными проблемами были выделение разумного кол-ва столбцов и сложности с делениями данных.

В поисках максимальной эффективности наша команда приняла решение создание создания общих столбцов для всех данных.
Это решение позволяет удалиться от монотонного текста приблизившись к параметрическим данным

Сами параметры разделялись путём выделения отдельных токенов


In [ ]:
for i in range (10): mtr[str(i+1) + '_parameter'] = ''

In [ ]:
for index, row in pd.DataFrame(mtr[:10]).iterrows():
    # do something with the row
    mas = row['Параметры'].split()
    for i, value in enumerate(mas[:10]):
      mtr.loc[index, f'{i+1}_parameter'] = value



In [ ]:
mtr.head(3)

,код СКМТР,Наименование,Маркировка,Регламенты (ГОСТ/ТУ),Параметры,Базисная,ОКПД2,group,1_parameter,2_parameter,3_parameter,4_parameter,5_parameter,6_parameter,7_parameter,8_parameter,9_parameter,10_parameter
0,8559150569,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-182 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,сорочка,124-108-182,(2-Я,ПОЛНОТНАЯ,ГРУППА),СРЕДНЯЯ,КАТЕГОРИЯ,ДОЛЖНОСТИ,2,ГАЛУНА,4
1,8559150570,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-188 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,сорочка,124-108-188,(2-Я,ПОЛНОТНАЯ,ГРУППА),СРЕДНЯЯ,КАТЕГОРИЯ,ДОЛЖНОСТИ,2,ГАЛУНА,4
2,8559150571,СОРОЧКА МУЖСКАЯ ФОРМЕННАЯ ПОВСЕДНЕВНАЯ С КОРОТ...,АО ФПК,ТУ 8559-004-00083262-2009,124-108-194 (2-Я ПОЛНОТНАЯ ГРУППА) СРЕДНЯЯ КАТ...,796,14.12.30.131,сорочка,124-108-194,(2-Я,ПОЛНОТНАЯ,ГРУППА),СРЕДНЯЯ,КАТЕГОРИЯ,ДОЛЖНОСТИ,2,ГАЛУНА,4


In [ ]:
mtr.to_csv('output.csv', sep=';')